# Entrainer YOLOv5 sur des abeilles
Dans ce projet nous allons utiliser yolo version 5 de "ultralytics repository" accessible depuis https://github.com/ultralytics/yolov5 appliquer à notre dataset qui constitue les deux classes _"bee"_ et _"queen"_ afin d'identifier et de localiser la reine d'abeille au milieu de sa colonie.
Pour entrainer notre detecteur voici les étapes que l'on va suivre dans ce tutoriel:

* Installer les dependences de YOLOv5 
* Télécharger notre YOLOv5 objet detection personaliser depuis notre repertoire github
* Ecrire notre YOLOv5 configuration d'apprentissage
* Executer l'entrainement YOLOv5
* Evaluer la performance de notre detection par YOLOv5 
* Exporter le YOLOv5 poids(weights) sur google drive




#Téléchargement des données et installations des dépendences


In [4]:
!pwd

/content/QueenBee_Training/yolov5


In [1]:
# Export code snippet and paste here
!git clone "https://github.com/ferio06/QueenBee_Training.git"
%cd QueenBee_Training/
!unzip DataBaseQueen.zip; rm DataBaseQueen.zip
!unzip yolov5.zip; rm yolov5.zip
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5
import torch
from IPython.display import Image, clear_output  # pour afficher les images
from utils.google_utils import gdrive_download  # pour télécharger les models/datasets

clear_output()

print('Ici on utilise torch %s \n \nMateriel utiliser : \n   > %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Ici on utilise torch 1.8.1+cu101 
 
Materiel utiliser : 
   > _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
# Voici le fichier YAML que Roboflow a écrit contenant les informations sur les classes de notre dataset
%cat ../data.yaml

train: ../train/images
val: ../valid/images

nc: 2
names: ['bee', 'queen']

# Definir la modèle de configuration et architecture

Ici on va écrire un script yaml script definissant les parametres pour notre modèle tel que le nombre de classe, anchors, et chaque couche.

In [ ]:
# définir le nombre de classe basé sur le fichier YAML
import yaml
with open("../data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#Pour ce projet nous allons utilisé le modèle de configuration suivant  
%cat /content/QueenBee_Training/yolov5/models/yolov5x.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-

In [ ]:
#Personnaliser iPython writefile pour nous permetre d'écrire nos variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [ ]:
%%writetemplate /content/QueenBee_Training/yolov5/models/custom_yolov5s.yaml

# parametres
nc: 2  # nombre de classe
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Entrainer notre détecteur YOLOv5 

C'est ici qu'on fait passer les nombres d'arguments :
- **img:** définit la taille d'image d'entrer
- **batch:** détermine la taille du batch 
- **epochs:** definit le numbre d' epochs d'entrainement. 
- **data:** specifier le chemin pour notre fichier yaml
- **cfg:** specifier notre model de configuration
- **weights:** specifier le chemin d'accès que l'on a personnaliser au weights. 
- **name:** nom donné au resultat
- **nosave:** sauvegarder seulement le checkpoint finale.
- **cache:** la cache des images pour une vitesse d'entrainement plus rapide.

In [ ]:
%%time
%cd /content/QueenBee_Training/yolov5/
!python train.py --img 416 --batch 5 --epochs 200 --data '../data.yaml' --cfg ./models/yolov5x.yaml --weights '' --name yolov5x_50_200  --cache

# Evaluation de la performance de notre detecteur YOLOv5

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
%cd /content/QueenBee_Training/yolov5/
!python detect.py --weights runs/exp0_yolov5x_15_200/weights/last.pt --img 416 --conf 0.4 --source ../test/images


In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/QueenBee_Training/yolov5/inference/output/*.jpg'): 
    display(Image(filename=imageName))
    print("\n")